# Confidence Interval & BootStraping Exercise
Compute confidence/credible intervals based on the four methods above for simulated data sampled from a population that is Gaussian distributed with mean 
=10 and standard deviation 
=2, for n=5, 10, 20, 40, 80, 160, 1000 at a 95% confidence level.

In [1]:
## SEM

import numpy as np
import pandas as pd
from scipy import stats

def simulate_intervals(mu=10.0, sigma=2.0, ns=[5,10,20,40,80,160,1000],
                       reps=5000, alpha=0.05, random_state=42):
    """
    Simulate confidence intervals for samples drawn from N(mu, sigma^2).
    
    Returns a pandas DataFrame with mean interval widths and coverage rates
    for both z-intervals (known sigma) and t-intervals (unknown sigma).
    """
    np.random.seed(random_state)
    results = []

    for n in ns:
        # z critical value
        z_crit = stats.norm.ppf(1 - alpha/2)

        z_contains = []
        t_contains = []
        z_widths = []
        t_widths = []

        for _ in range(reps):
            samp = np.random.normal(loc=mu, scale=sigma, size=n)
            m = samp.mean()
            s = samp.std(ddof=1)

            # z-interval (known sigma)
            sem_known = sigma / np.sqrt(n)
            z_lo, z_hi = m - z_crit*sem_known, m + z_crit*sem_known
            z_contains.append(z_lo <= mu <= z_hi)
            z_widths.append(z_hi - z_lo)

            # t-interval (unknown sigma, estimated from sample)
            t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
            sem_est = s / np.sqrt(n)
            t_lo, t_hi = m - t_crit*sem_est, m + t_crit*sem_est
            t_contains.append(t_lo <= mu <= t_hi)
            t_widths.append(t_hi - t_lo)

        results.append({
            "n": n,
            "mean_width_z": np.mean(z_widths),
            "coverage_z": np.mean(z_contains),
            "mean_width_t": np.mean(t_widths),
            "coverage_t": np.mean(t_contains)
        })

    return pd.DataFrame(results)

# Run simulation
df = simulate_intervals()

print(df)

      n  mean_width_z  coverage_z  mean_width_t  coverage_t
0     5      3.506090      0.9602      4.674692      0.9524
1    10      2.479180      0.9392      2.779365      0.9434
2    20      1.753045      0.9502      1.846030      0.9496
3    40      1.239590      0.9502      1.272158      0.9512
4    80      0.876523      0.9486      0.887775      0.9482
5   160      0.619795      0.9544      0.624586      0.9542
6  1000      0.247918      0.9562      0.248116      0.9566


In [2]:
## t-distribution

import numpy as np
import pandas as pd
from scipy import stats

def simulate_intervals_tdist(mu=10.0, sigma=2.0, df=5,
                             ns=[5,10,20,40,80,160,1000],
                             reps=5000, alpha=0.05, random_state=42):
    """
    Simulate confidence intervals for samples drawn from a Student-t distribution
    with given degrees of freedom (df), shifted to mean=mu and scaled to stdev=sigma.
    
    Returns a pandas DataFrame with mean interval widths and coverage rates
    for both z-intervals (assuming known sigma) and t-intervals (estimating sigma).
    """
    np.random.seed(random_state)
    results = []

    # scale factor so that the t(df) distribution has stdev = sigma
    t_std = np.sqrt(df / (df - 2)) if df > 2 else np.inf
    scale = sigma / t_std

    for n in ns:
        z_crit = stats.norm.ppf(1 - alpha/2)

        z_contains, t_contains = [], []
        z_widths, t_widths = [], []

        for _ in range(reps):
            # sample from Student-t(df), then rescale + shift
            samp = stats.t.rvs(df, size=n) * scale + mu
            m = samp.mean()
            s = samp.std(ddof=1)

            # z-interval (pretending sigma is known)
            sem_known = sigma / np.sqrt(n)
            z_lo, z_hi = m - z_crit*sem_known, m + z_crit*sem_known
            z_contains.append(z_lo <= mu <= z_hi)
            z_widths.append(z_hi - z_lo)

            # t-interval (estimate sigma from sample)
            t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
            sem_est = s / np.sqrt(n)
            t_lo, t_hi = m - t_crit*sem_est, m + t_crit*sem_est
            t_contains.append(t_lo <= mu <= t_hi)
            t_widths.append(t_hi - t_lo)

        results.append({
            "n": n,
            "mean_width_z": np.mean(z_widths),
            "coverage_z": np.mean(z_contains),
            "mean_width_t": np.mean(t_widths),
            "coverage_t": np.mean(t_contains)
        })

    return pd.DataFrame(results)

# Example run
df_results = simulate_intervals_tdist(mu=10, sigma=2, df=5)
print(df_results)

      n  mean_width_z  coverage_z  mean_width_t  coverage_t
0     5      3.506090      0.9534      4.475024      0.9564
1    10      2.479180      0.9394      2.718313      0.9456
2    20      1.753045      0.9484      1.814195      0.9516
3    40      1.239590      0.9516      1.258287      0.9526
4    80      0.876523      0.9534      0.882558      0.9532
5   160      0.619795      0.9494      0.621618      0.9504
6  1000      0.247918      0.9480      0.248067      0.9460


In [3]:
## Bootstraped Confidence Intervals

import numpy as np
import pandas as pd
from scipy import stats

def simulate_bootstrap_intervals(mu=10.0, sigma=2.0, 
                                 ns=[5,10,20,40,80,160,1000],
                                 reps=2000,  # fewer reps because bootstrapping is expensive
                                 n_boot=2000,
                                 alpha=0.05,
                                 random_state=42):
    """
    Simulate bootstrap confidence intervals for samples from N(mu, sigma^2).
    
    For each sample:
      - Draw n_boot bootstrap resamples
      - Compute the sample mean for each
      - Use the percentile method to get a 100*(1-alpha)% CI
    
    Returns a DataFrame summarizing mean interval width and coverage.
    """
    np.random.seed(random_state)
    results = []

    for n in ns:
        widths = []
        contains = []

        for _ in range(reps):
            # original sample
            samp = np.random.normal(loc=mu, scale=sigma, size=n)

            # bootstrap resamples
            boot_means = []
            for _ in range(n_boot):
                resample = np.random.choice(samp, size=n, replace=True)
                boot_means.append(np.mean(resample))
            boot_means = np.array(boot_means)

            # percentile interval
            lo, hi = np.percentile(boot_means, [100*alpha/2, 100*(1-alpha/2)])
            widths.append(hi - lo)
            contains.append(lo <= mu <= hi)

        results.append({
            "n": n,
            "mean_width_boot": np.mean(widths),
            "coverage_boot": np.mean(contains)
        })

    return pd.DataFrame(results)

# Example run
df_boot = simulate_bootstrap_intervals()
print(df_boot)


KeyboardInterrupt: 

In [5]:
## Bayesian Credible Intervals

import numpy as np
import pandas as pd
from scipy import stats

def simulate_bayesian_credible_intervals(
    mu=10.0,
    sigma=2.0,
    ns=[5,10,20,40,80,160,1000],
    reps=5000,
    alpha=0.05,
    random_state=42,
    dist='normal',   # 'normal' or 't'
    t_df=5           # if dist=='t', degrees of freedom for population
):
    """
    Simulate Bayesian credible intervals for the mean under two scenarios:
      (A) known sigma (posterior Normal under flat prior for mu)
      (B) unknown sigma with noninformative prior -> posterior t for mu

    Returns a DataFrame summarizing mean interval width and empirical coverage.
    """
    np.random.seed(random_state)
    results = []

    z_crit = stats.norm.ppf(1 - alpha/2)

    for n in ns:
        widths_known = []
        contains_known = []

        widths_unknown = []
        contains_unknown = []

        for _ in range(reps):
            # draw a sample from chosen population
            if dist == 'normal':
                sample = np.random.normal(loc=mu, scale=sigma, size=n)
            elif dist == 't':
                # scale t(df) so that its SD = sigma (if df>2)
                if t_df <= 2:
                    raise ValueError("t_df must be >2 to have finite variance")
                t_std = np.sqrt(t_df / (t_df - 2))
                sample = stats.t.rvs(t_df, size=n) * (sigma / t_std) + mu
            else:
                raise ValueError("dist must be 'normal' or 't'")

            xbar = sample.mean()
            s = sample.std(ddof=1)

            # --- (A) Known sigma: posterior N(xbar, sigma/sqrt(n)) with flat prior ---
            post_sd_known = sigma / np.sqrt(n)
            lo_known = xbar - z_crit * post_sd_known
            hi_known = xbar + z_crit * post_sd_known
            widths_known.append(hi_known - lo_known)
            contains_known.append((lo_known <= mu) and (mu <= hi_known))

            # --- (B) Unknown sigma: marginal posterior of mu is t_{n-1}(xbar, s/sqrt(n)) ---
            t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
            post_scale = s / np.sqrt(n)
            lo_unk = xbar - t_crit * post_scale
            hi_unk = xbar + t_crit * post_scale
            widths_unknown.append(hi_unk - lo_unk)
            contains_unknown.append((lo_unk <= mu) and (mu <= hi_unk))

        results.append({
            "n": n,
            "mean_width_known_sigma": np.mean(widths_known),
            "coverage_known_sigma": np.mean(contains_known),
            "mean_width_unknown_sigma": np.mean(widths_unknown),
            "coverage_unknown_sigma": np.mean(contains_unknown)
        })

    return pd.DataFrame(results)

# Example runs ---------------------------------------------------------
if __name__ == "__main__":
    ns = [5,10,20,40,80,160,1000]

    # 1) Data generated from Gaussian N(10, 2^2)
    df_norm = simulate_bayesian_credible_intervals(
        mu=10.0, sigma=2.0, ns=ns, reps=5000, dist='normal', random_state=123
    )
    print("Population: Normal(mu=10, sigma=2)\n", df_norm, "\n")

    # 2) Data generated from heavy-tailed t (df=5), scaled to have sd ~ 2
    df_t = simulate_bayesian_credible_intervals(
        mu=10.0, sigma=2.0, ns=ns, reps=5000, dist='t', t_df=5, random_state=123
    )
    print("Population: scaled Student-t(df=5) with sd~2\n", df_t)

    # optionally save
    df_norm.to_csv("bayes_credible_intervals_normal_population.csv", index=False)
    df_t.to_csv("bayes_credible_intervals_t5_population.csv", index=False)


Population: Normal(mu=10, sigma=2)
       n  mean_width_known_sigma  coverage_known_sigma  \
0     5                3.506090                0.9456   
1    10                2.479180                0.9550   
2    20                1.753045                0.9474   
3    40                1.239590                0.9480   
4    80                0.876523                0.9478   
5   160                0.619795                0.9464   
6  1000                0.247918                0.9472   

   mean_width_unknown_sigma  coverage_unknown_sigma  
0                  4.641488                  0.9472  
1                  2.795783                  0.9558  
2                  1.842620                  0.9482  
3                  1.269930                  0.9462  
4                  0.887983                  0.9504  
5                  0.623434                  0.9474  
6                  0.248196                  0.9470   

Population: scaled Student-t(df=5) with sd~2
       n  mean_width_known_s

OSError: [Errno 30] Read-only file system: 'bayes_credible_intervals_normal_population.csv'